# API Demo

Simple demo of how the API should be contacted, showing how to get an authentication token and then use it to get or post data

In [1]:
import requests
import os

# load the correct host name from the environment if defined
HOSTNAME = os.getenv('API_HOSTNAME', 'localhost')
PORT = 443
BASE_URL = f'https://{HOSTNAME}:{PORT}'

YOUR_USERNAME = 'sysadmin'# change this, until topic 7 gives you real credentials anything will work
YOUR_PASSWORD = 'sysadmin'# change this, until topic 7 gives you real credentials anything will work

## Get an authentication token

In [2]:
AUTH_URL = f'{BASE_URL}/user/login'

auth_token = None

def get_token(url, username, password):
    headers = {
        'username': username,
        'password': password
    }  
    response = requests.post(url, headers=headers, verify=False)
    if response.status_code == 200:
        print('You are authenticated!')
        return response.json()
    elif response.status_code == 400:
        print('Wrong credentials')
    elif response.status_code == 500:
        print('Connection error')
    else:
        print('Unknown error')
    return None

auth_token = get_token(AUTH_URL, YOUR_USERNAME, YOUR_PASSWORD)
print('Token: ', auth_token)

You are authenticated!
Token:  Bearer test_token


c:\Users\andre\anaconda3\envs\HLT\lib\site-packages\urllib3\connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


## Get machine list

When you call an API method, you need the authentication token.
The token may be expired if some time has passed since you got it, so you should get a new one before calling the API if you get an authentication error.

In [6]:
def call_with_token(url, token, call_type='GET', header_params=None):
    headers = {} if header_params is None else header_params
    headers['Authorization'] = token
    if call_type == 'GET':
        response = requests.get(url, headers=headers, verify=False)
    elif call_type == 'POST':
        response = requests.post(url, headers=headers, verify=False)
    elif call_type == 'PUT':
        response = requests.put(url, headers=headers, verify=False)
    elif call_type == 'DELETE':
        response = requests.delete(url, headers=headers, verify=False)
    else:
        print('Unknown call type')
        return None, None
    if response.status_code == 500:
        print('Server error: ', response.reason)
        return None, response.status_code
    elif response.status_code == 400:
        print('Bad request: ', response.reason)
        return None, response.status_code
    elif response.status_code == 401:
        print('Unauthorized')
        return None, response.status_code
    elif response.status_code == 200:
        return response.json(), response.status_code
    else:
        print('Unknown error: ', response.reason)
        return None, response.status_code
    
def call_and_retry(url, token, call_type='GET', header_params=None):
    response, status = call_with_token(url, token, call_type, header_params)
    if status == 401:
        print('Token expired, getting new token')
        token = get_token(AUTH_URL, YOUR_USERNAME, YOUR_PASSWORD)
        if token is None:
            print('Failed to get new token')
            return None, 401
        return call_and_retry(url, token, call_type, header_params)
    return response, status

In [4]:
#get machine list
MACHINE_URL = f'{BASE_URL}/machine'
machines, status = call_and_retry(MACHINE_URL, auth_token)
if status == 200:
    print('Machines: ', machines)
else:
    print('Failed to get machines')

Machines:  [{'factory': 'Factory 1', 'id': 2, 'machineType': 'Type Y', 'name': 'Machine B', 'productionLine': 2, 'status': 'Down'}, {'factory': 'Factory 2', 'id': 3, 'machineType': 'Type X', 'name': 'Machine C', 'productionLine': 1, 'status': 'Down'}, {'factory': 'Factory 2', 'id': 4, 'machineType': 'Type Z', 'name': 'Machine D', 'productionLine': 3, 'status': 'Down'}, {'factory': 'Factory X', 'id': 1, 'machineType': 'Laser welding', 'name': 'Laser Welding Machine 10', 'productionLine': 2, 'status': 'Down'}, {'factory': 'Factory X', 'id': 7, 'machineType': 'Welding Machine', 'name': 'KB test machine', 'productionLine': 2, 'status': 'Up'}, {'factory': 'Factory X', 'id': 8, 'machineType': 'Cutting Machine', 'name': 'KB test machine 2', 'productionLine': 2, 'status': 'Down'}]


c:\Users\andre\anaconda3\envs\HLT\lib\site-packages\urllib3\connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


## Add an alert

In [ ]:
ENDPOINT_URL = f'{BASE_URL}/alert'
HEADERS = {
    "timestamp": "2023-10-01 08:00:00",
    "machineID": "1",
    "severity": "low",
    "description": "This is a test alert",
    "KPI": "Temperature"
}
res, status = call_and_retry(ENDPOINT_URL, auth_token, call_type='POST', header_params=HEADERS)
if status == 200:
    print('Alert added')
else:
    print('Failed to add alerts')